"""
core2_05_curieus_queue_export.ipynb

목적:
- Curieus를 '결정 도구'가 아니라 '관측 대상 생성기'로만 사용한다.
- Core 2에서 발생한 mutation 중,
  예측 기반 개입이 발생한 지점들을
  Curieus 분석 대기열(queue)로 정리한다.

중요:
- Curieus 결과는 Core 2/3의 의사결정에 사용하지 않는다.
- Curieus는 '왜 흔들렸는지 관측'하기 위한 외부 관측 장치이다.
"""

In [2]:
import pandas as pd
from pathlib import Path
import uuid

# Core 2 산출물 경로
artifact_dir = Path("../artifact/core2")

prediction_trace_path = artifact_dir / "prediction_trace.csv"
mutation_applied_path = artifact_dir / "mutation_applied_log.csv"

prediction_trace = pd.read_csv(prediction_trace_path)
mutation_applied = pd.read_csv(mutation_applied_path)

prediction_trace.head(), mutation_applied.head()

(  antibody_key  step                                   sequence_current  \
 0    GDPa1-001     1  EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...   
 1    GDPa1-001     2  EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...   
 2    GDPa1-001     3  EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...   
 3    GDPa1-001     4  EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...   
 4    GDPa1-001     5  EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...   
 
    pred_score  pred_score_delta decision mutation_id_applied  \
 0    2.013943          0.013943   MUTATE    GDPa1-001_mut0_1   
 1    1.966444         -0.047499     HOLD                 NaN   
 2    1.943947         -0.022497     HOLD                 NaN   
 3    1.916268         -0.027679     HOLD                 NaN   
 4    1.939915          0.023647   MUTATE    GDPa1-001_mut1_5   
 
    intervention_count_cum  
 0                       1  
 1                       1  
 2                       1  
 3                       1  
 4   

Curieus 관측 대상으로 삼을 step 정의

정책
	•	mutation이 실제로 발생한 step
	•	즉, decision == "MUTATE" 이고
	•	mutation_id_applied != null 인 경우

In [3]:
# mutation이 실제로 적용된 step만 추출
curieus_targets = prediction_trace[
    prediction_trace["mutation_id_applied"].notna()
].copy()

curieus_targets.head()

,antibody_key,step,sequence_current,pred_score,pred_score_delta,decision,mutation_id_applied,intervention_count_cum
0,GDPa1-001,1,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,2.013943,0.013943,MUTATE,GDPa1-001_mut0_1,1
4,GDPa1-001,5,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.939915,0.023647,MUTATE,GDPa1-001_mut1_5,2
5,GDPa1-001,6,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPEPNYKCRV...,1.957585,0.017670,MUTATE,GDPa1-001_mut2_6,3
6,GDPa1-001,7,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.996803,0.039218,MUTATE,GDPa1-001_mut1_7,4
11,GDPa1-001,12,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.873068,0.000536,MUTATE,GDPa1-001_mut1_12,5


	•	Curieus는 판단 요청이 아니라 관측 요청
	•	reason은 항상 정책 트리거 기반임을 명시

In [4]:
curieus_queue = pd.DataFrame({
    "antibody_key": curieus_targets["antibody_key"],
    "step": curieus_targets["step"],
    "sequence_after": curieus_targets["sequence_current"],
    "mutation_id_applied": curieus_targets["mutation_id_applied"],
    "reason": "triggered_by_pred_worsening",
    "curieus_job_id": [None] * len(curieus_targets)  # 나중에 채움
})

curieus_queue.head() # Curieus Queue 포맷으로 정규화

,antibody_key,step,sequence_after,mutation_id_applied,reason,curieus_job_id
0,GDPa1-001,1,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut0_1,triggered_by_pred_worsening,None
4,GDPa1-001,5,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut1_5,triggered_by_pred_worsening,None
5,GDPa1-001,6,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPEPNYKCRV...,GDPa1-001_mut2_6,triggered_by_pred_worsening,None
6,GDPa1-001,7,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut1_7,triggered_by_pred_worsening,None
11,GDPa1-001,12,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut1_12,triggered_by_pred_worsening,None


🔹 이 ID는 분석 요청 식별자일 뿐
🔹 의사결정과는 무관

In [5]:
# 선택: Curieus job id를 미리 생성하고 싶을 경우
import uuid

curieus_queue["curieus_job_id"] = [
    f"curieus_job_{uuid.uuid4().hex[:8]}"
    for _ in range(len(curieus_queue))
]

curieus_queue.head()

,antibody_key,step,sequence_after,mutation_id_applied,reason,curieus_job_id
0,GDPa1-001,1,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut0_1,triggered_by_pred_worsening,curieus_job_5e0bd1aa
4,GDPa1-001,5,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut1_5,triggered_by_pred_worsening,curieus_job_50cee759
5,GDPa1-001,6,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPEPNYKCRV...,GDPa1-001_mut2_6,triggered_by_pred_worsening,curieus_job_01301abb
6,GDPa1-001,7,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut1_7,triggered_by_pred_worsening,curieus_job_bf48616a
11,GDPa1-001,12,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,GDPa1-001_mut1_12,triggered_by_pred_worsening,curieus_job_d0b4a3b0


이 셀의 의미:
	•	특정 항체만 과도하게 queue에 쌓였는지
	•	Core 2의 불안정성이 관측 요청 증가로 직결되는지

In [ ]:
queue_distribution = curieus_queue.groupby("antibody_key").size()

queue_distribution

antibody_key
GDPa1-001     9
GDPa1-045    14
GDPa1-183     6
dtype: int64

In [8]:
from pathlib import Path

artifact_dir = Path("../artifact/core2")
artifact_dir.mkdir(parents=True, exist_ok=True)

output_path = artifact_dir / "curieus_queue.csv"
curieus_queue.to_csv(output_path, index=False)

output_path

PosixPath('../artifact/core2/curieus_queue.csv')

print("""
[Core 2 → Curieus 연결 선언]

- Curieus는 이 프로젝트에서 '결정자'가 아니다.
- Curieus는 예측 기반 설계가 흔들린 지점을 관측하기 위한 외부 분석 장치이다.
- curieus_queue.csv는 판단 요청이 아니라 관측 요청 목록이다.
""")